# Laboratorio 5 - Python en Paralelo
**alumno09:** Laura Llamas López

## Reduction: the sum of the elements of an array

In [ ]:
# Configuración para ejecución desde línea de comandos
import sys

value = 5*10**7  # Valor por defecto

if len(sys.argv) > 1:
    try:
        value = int(sys.argv[1])
    except ValueError:
        print(f"Error: argumento '{sys.argv[1]}' no válido. Usando valor por defecto.")

if len(sys.argv) > 2:
    ncores = int(sys.argv[2])
else:
    ncores = None  # Para ejecución interactiva, usará el bucle [2, 4]

print(f"Ejecutando con value = {value:,}")

In [4]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

print("="*70)
print("CÓDIGO ORIGINAL (SECUENCIAL)")
print("="*70)

X = np.random.rand(value)

# Para imprimir los primeros valores del array

#print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")

# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


CÓDIGO ORIGINAL (SECUENCIAL)
Time taken by reduction operation using a function: 4.79 s ± 39.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 25001243.652032945

Time taken by reduction operation using numpy.sum(): 19 ms ± 11.4 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 25001243.652032144 
 


## a) Paralelización con multiprocessing (Pool)

In [3]:
from multiprocessing import Pool

def reduc_operation_parallel(A, n_processes=4):
    """
    Compute the sum of the elements of Array A using multiprocessing.
    
    Parameters:
    -----------
    A : numpy array
        Array to sum
    n_processes : int
        Number of processes to use
    
    Returns:
    --------
    float : Sum of all elements
    """
    # Paso 1: Dividir el array en tantos sub-arrays como procesos
    chunks = np.array_split(A, n_processes)
    
    # Paso 2: Crear el pool de procesos
    with Pool(processes=n_processes) as pool:
        # Paso 3: Usar map para enviar cada sub-array a la función
        partial_results = pool.map(reduc_operation, chunks)
    
    # Paso 4: Reducir (sumar) los resultados parciales
    total_sum = sum(partial_results)
    
    return total_sum


print("="*70)
print("MULTIPROCESSING CON POOL")
print("="*70)

# Probar con diferentes números de procesos
if ncores:
    print(f"\n>>> Con {ncores} procesos:")
    tiempo = %timeit -r 2 -o -q reduc_operation_parallel(X, n_processes=ncores)
    print(f"    Time: {tiempo}")
    print(f"    Result: {reduc_operation_parallel(X, n_processes=ncores)}")
else:
    for n_procs in [2, 4]:
        print(f"\n>>> Con {n_procs} procesos:")
        tiempo = %timeit -r 2 -o -q reduc_operation_parallel(X, n_processes=n_procs)
        print(f"    Time: {tiempo}")
        print(f"    Result: {reduc_operation_parallel(X, n_processes=n_procs)}")

MULTIPROCESSING CON POOL

>>> Con 2 procesos:
    Time: 3.31 s ± 11.7 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 25001520.247405216

>>> Con 4 procesos:
    Time: 1.82 s ± 1.91 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 25001520.247409306


### Interpretación de resultados: Código Original vs Multiprocessing

**Código Original (Secuencial):**
- `reduc_operation()`: **4.79 s**
- `numpy.sum()`: **19 ms** (252x más rápido que la función manual)

**Multiprocessing con Pool:**
- 2 procesos: **3.31 s** → Speedup: **1.45x** (respecto a 4.79 s)
- 4 procesos: **1.82 s** → Speedup: **2.63x** (respecto a 4.79 s)

La paralelización funciona, ya que al usar 2 procesos reducimos el tiempo a ~69% del original, y con 4 procesos a ~38%. No hay speedup lineal, con 4 procesos solo se obtiene 2.63x.

**Verdadero problema**: Incluso con 4 procesos, sigue siendo ~96x más lento que `numpy.sum()`. Esto indica que el cuello de botella real es la función `reduc_operation()` sin optimizar, que usa un bucle Python puro muy ineficiente. En conclusión, Multiprocessing mejora el rendimiento de código Python puro, pero no resuelve el problema de fondo: necesitamos optimizar la función base con Numba.

## b) Optimización y paralelización con Numba

In [6]:
from numba import njit, prange, set_num_threads

# Versión 1: Numba secuencial con @njit
@njit
def reduc_operation_numba(A):
    """Compute the sum of the elements of Array A - optimized with Numba."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Versión 2: Numba paralelo con @njit(parallel=True) y prange
@njit(parallel=True)
def reduc_operation_numba_parallel(A):
    """Compute the sum of the elements of Array A - parallelized with Numba."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s


print("="*70)
print("NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN")
print("="*70)

# Warmup: compilar las funciones antes de medir tiempos
_ = reduc_operation_numba(X)
_ = reduc_operation_numba_parallel(X)

# Versión secuencial con Numba
print("\n>>> Numba secuencial (@njit):")
tiempo_numba = %timeit -r 2 -o -q reduc_operation_numba(X)
print(f"    Time: {tiempo_numba}")
print(f"    Result: {reduc_operation_numba(X)}")

# Versión paralela con Numba - probar con diferentes números de threads
print("\n>>> Numba paralelo (@njit(parallel=True) con prange):")

if ncores:
    # Ejecución desde SLURM con ncores específico
    set_num_threads(ncores)
    print(f"\n    Con {ncores} threads:")
    tiempo = %timeit -r 2 -o -q reduc_operation_numba_parallel(X)
    print(f"        Time: {tiempo}")
    print(f"        Result: {reduc_operation_numba_parallel(X)}")
else:
    # Ejecución interactiva con bucle
    for n_threads in [2, 4]:
        set_num_threads(n_threads)
        print(f"\n    Con {n_threads} threads:")
        tiempo = %timeit -r 2 -o -q reduc_operation_numba_parallel(X)
        print(f"        Time: {tiempo}")
        print(f"        Result: {reduc_operation_numba_parallel(X)}")

NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN

>>> Numba secuencial (@njit):
    Time: 50.2 ms ± 8.92 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
    Result: 25000425.97907481

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 2 threads:
        Time: 25.4 ms ± 6.27 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
        Result: 25000425.97907329

    Con 4 threads:
        Time: 13.5 ms ± 4.66 μs per loop (mean ± std. dev. of 2 runs, 100 loops each)
        Result: 25000425.97907128


### Interpretación de resultados: Numba vs Métodos Anteriores

**Numba - Optimización:**
- Numba secuencial `@njit`: **50.2 ms** → Speedup: **95.4x** (respecto a 4.79 s original)

**Numba - Paralelización con prange:**
- 2 threads: **25.4 ms** → Speedup: **1.98x** (respecto a Numba secuencial)
- 4 threads: **13.5 ms** → Speedup: **3.72x** (respecto a Numba secuencial)

La optimización con Numba es altamente eficiente: con solo añadir `@njit` conseguimos una mejora de ~95x. Además, con `@njit(parallel=True)` y 4 threads obtenemos una mejora de ~355x respecto al código original (4.79s → 13.5ms).

**Comparación con numpy.sum() (19 ms)**: Numba paralelo con 4 threads (13.5 ms) es incluso 1.4x más rápido que numpy. Esto demuestra que Numba puede igualar o superar el rendimiento de bibliotecas altamente optimizadas.

**Comparación con Multiprocessing**: Numba paralelo (13.5 ms) es 135x más rápido que multiprocessing con 4 procesos (1.82 s). En conclusión, para operaciones intensivas en cómputo con arrays numéricos, **Numba es la mejor opción**.

## d) Resultados de ejecución en cola mendel

### Para 10^8

```
========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=100000000 | NCORES=1
========================================
Ejecutando con value = 100,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 17.3 s ± 78.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 49996835.05663532

Time taken by reduction operation using numpy.sum(): 71.9 ms ± 6.66 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 49996835.05662809 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 1 procesos:
    Time: 23.3 s ± 24.4 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 49996835.05663532
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 115 ms ± 10.4 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
    Result: 49996835.05663532

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 1 threads:
        Time: 115 ms ± 107 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
        Result: 49996835.05663532

========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=100000000 | NCORES=2
========================================
Ejecutando con value = 100,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 17.3 s ± 51.3 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 49998103.681402504

Time taken by reduction operation using numpy.sum(): 64.1 ms ± 2.56 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 49998103.68139622 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 2 procesos:
    Time: 11.1 s ± 74.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 49998103.681397445
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 115 ms ± 13.2 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
    Result: 49998103.681402504

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 2 threads:
        Time: 57.8 ms ± 3.57 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
        Result: 49998103.681397445

========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=100000000 | NCORES=4
========================================
Ejecutando con value = 100,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 17.7 s ± 350 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 50002788.0551457

Time taken by reduction operation using numpy.sum(): 76.6 ms ± 2.29 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 50002788.05516134 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 4 procesos:
    Time: 6.62 s ± 45.7 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 50002788.05515565
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 116 ms ± 136 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
    Result: 50002788.0551457

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 4 threads:
        Time: 42.9 ms ± 7.89 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
        Result: 50002788.05515565

========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=100000000 | NCORES=8
========================================
Ejecutando con value = 100,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 17.7 s ± 102 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 50001225.412749276

Time taken by reduction operation using numpy.sum(): 77.8 ms ± 1.97 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 50001225.41274719 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 8 procesos:
    Time: 3.74 s ± 59.8 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 50001225.41274643
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 116 ms ± 160 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
    Result: 50001225.412749276

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 8 threads:
        Time: 34.9 ms ± 1.05 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)
        Result: 50001225.41274643
```

### Para 10^9

```
========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=1000000000 | NCORES=1
========================================
Ejecutando con value = 1,000,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 2min 52s ± 609 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 499983134.96144164

Time taken by reduction operation using numpy.sum(): 648 ms ± 165 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 499983134.96098346 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 1 procesos:
    Time: 3min 29s ± 113 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 499983134.96144164
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 1.16 s ± 122 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 499983134.96144164

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 1 threads:
        Time: 1.16 s ± 937 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
        Result: 499983134.96144164

========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=1000000000 | NCORES=2
========================================
Ejecutando con value = 1,000,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 2min 50s ± 1.2 s per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 500000865.6916198

Time taken by reduction operation using numpy.sum(): 658 ms ± 17.7 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 500000865.6917266 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 2 procesos:
    Time: 1min 51s ± 46.4 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 500000865.6916547
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 1.15 s ± 110 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 500000865.6916198

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 2 threads:
        Time: 577 ms ± 32.3 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
        Result: 500000865.6916547

========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=1000000000 | NCORES=4
========================================
Ejecutando con value = 1,000,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 2min 53s ± 796 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 499998410.89959055

Time taken by reduction operation using numpy.sum(): 758 ms ± 39.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 499998410.89968926 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 4 procesos:
    Time: 1min 2s ± 136 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 499998410.899746
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 1.15 s ± 1.44 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 499998410.89959055

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 4 threads:
        Time: 298 ms ± 2.18 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
        Result: 499998410.89974606

========================================
Ejecutando reduc-operation-array-par-alumno09.ipynb
VALUE=1000000000 | NCORES=8
========================================
Ejecutando con value = 1,000,000,000
======================================================================
CÓDIGO ORIGINAL (SECUENCIAL)
======================================================================
Time taken by reduction operation using a function: 2min 50s ± 678 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 499996052.68939084

Time taken by reduction operation using numpy.sum(): 695 ms ± 3.45 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Now, the result using numpy.sum(): 499996052.6897362 
 
======================================================================
MULTIPROCESSING CON POOL
======================================================================

>>> Con 8 procesos:
    Time: 37 s ± 108 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 499996052.68976915
======================================================================
NUMBA: OPTIMIZACIÓN Y PARALELIZACIÓN
======================================================================

>>> Numba secuencial (@njit):
    Time: 1.15 s ± 1.11 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
    Result: 499996052.68939084

>>> Numba paralelo (@njit(parallel=True) con prange):

    Con 8 threads:
        Time: 233 ms ± 5.39 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
        Result: 499996052.68976915



## e) Comparación e interpretación de los tiempos obtenidos 

### Resultados para value = 10<sup>8</sup> 

| Método | 1 core | 2 cores | 4 cores | 8 cores |
|--------|--------|---------|---------|---------|
| Código original | 17.3 s | 17.3 s | 17.7 s | 17.7 s |
| numpy.sum() | 71.9 ms | 64.1 ms | 76.6 ms | 77.8 ms |
| Multiprocessing (Pool) | 23.3 s | 11.1 s | 6.62 s | 3.74 s |
| Numba secuencial | 115 ms | 115 ms | 116 ms | 116 ms |
| Numba prange (threads) | 115 ms | 57.8 ms | 42.9 ms | 34.9 ms |

### Resultados para value = 10<sup>9</sup> 

| Método | 1 core | 2 cores | 4 cores | 8 cores |
|--------|--------|---------|---------|---------|
| Código original | 172 s | 170 s | 173 s | 170 s |
| numpy.sum() | 648 ms | 658 ms | 758 ms | 695 ms |
| Multiprocessing (Pool) | 209 s | 111 s | 62 s | 37 s |
| Numba secuencial | 1.16 s | 1.15 s | 1.15 s | 1.15 s |
| Numba prange (threads) | 1.16 s | 577 ms | 298 ms | 233 ms |

### Interpretación de resultados

Al observar los resultados obtenidos en la ejecución sobre la cola mendel con arrays de gran tamaño, se pueden extraer varias conclusiones importantes sobre el rendimiento de las diferentes técnicas de paralelización aplicadas a operaciones de reducción.

En primer lugar, destaca la enorme diferencia de rendimiento entre el código original sin optimizar y las versiones mejoradas. Para un array de 10<sup>8</sup> elementos, el código secuencial tarda aproximadamente 17.3 segundos, mientras que con numpy.sum() este tiempo se reduce drásticamente a 71.9 milisegundos, lo que supone una mejora de más de 200 veces. Sin embargo, lo más sorprendente es que al aplicar Numba con el decorador @njit conseguimos tiempos de 115 milisegundos, una mejora de aproximadamente 150 veces respecto al código original, lo cual demuestra la potencia de la compilación JIT de Numba para optimizar bucles Python puros.

Cuando analizamos el comportamiento del paquete multiprocessing, observamos algo contraintuivo pero revelador: usar multiprocessing con un solo núcleo resulta más lento que el código original (23.3 segundos frente a 17.3 segundos). Esto se debe al overhead que introduce la creación de procesos, la división de datos y la comunicación entre procesos, sin obtener ningún beneficio de la paralelización. A medida que incrementamos el número de núcleos, multiprocessing empieza a mostrar mejoras: con 2 núcleos reduce el tiempo a 11.1 segundos, con 4 núcleos a 6.62 segundos, y con 8 núcleos alcanza los 3.74 segundos. Aunque hay una clara mejora, el speedup es sublineal (4.6x con 8 núcleos en lugar del ideal 8x) debido al overhead mencionado.

Por otro lado, Numba con paralelización automática mediante prange demuestra ser significativamente más eficiente. Partiendo de 115 milisegundos con la versión secuencial @njit, al activar parallel=True y usar 2 threads el tiempo se reduce a 57.8 ms, con 4 threads a 42.9 ms, y con 8 threads a 34.9 ms. Lo más notable es que con 8 threads, Numba paralelo es aproximadamente 107 veces más rápido que multiprocessing con la misma configuración (34.9 ms frente a 3.74 segundos), y además consigue superar incluso a numpy.sum(), que tarda alrededor de 72 milisegundos. Esto evidencia que Numba no solo optimiza el código mediante compilación JIT, sino que también gestiona la paralelización de manera mucho más eficiente que multiprocessing, con menor overhead y mejor aprovechamiento de los recursos.

Al escalar el problema a 10<sup>9</sup> elementos, las tendencias se mantienen pero se amplifican. El código original necesita casi 3 minutos (172 segundos) para completarse, lo cual justifica los largos tiempos de ejecución observados durante los experimentos. Numba secuencial tarda 1.16 segundos, y la versión paralela con 8 threads reduce este tiempo a solo 233 milisegundos, consiguiendo una mejora total de más de 700 veces respecto al código original. Multiprocessing con 8 núcleos tarda 37 segundos, que aunque es una mejora considerable respecto al código base, sigue siendo 159 veces más lento que Numba paralelo.

Es importante mencionar que tanto el código original como las versiones de Numba secuencial y numpy.sum() mantienen tiempos constantes independientemente del número de cores configurados, ya que no explotan la paralelización. En cambio, multiprocessing y Numba prange sí muestran claras mejoras al aumentar los cores, aunque Numba lo hace de forma mucho más eficiente.

En definitiva, estos resultados confirman que para operaciones matemáticas intensivas sobre arrays numéricos, Numba representa la mejor solución en términos de facilidad de implementación y rendimiento. Añadir simplemente @njit ya proporciona mejoras notables, y activar la paralelización con parallel=True y prange ofrece beneficios adicionales significativos sin la complejidad ni el overhead de multiprocessing. Multiprocessing sigue siendo útil para casos donde el código no es compatible con Numba o cuando se necesita paralelizar tareas más complejas, pero para cálculos numéricos puros, Numba es superior.